### **Extracción Data - Autos**

In [1]:
## Librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from tqdm import tqdm  # pip install tqdm
import datetime as dt
import re

In [5]:
# enlace = "https://neoauto.com/venta-de-autos-seminuevos"
enlace = "https://neoauto.com/venta-de-autos-nuevos?page=3"
# enlace = "https://neoauto.com/venta-de-autos-nuevos?page=1"

## Driver Setup

In [6]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")   # modo oculto
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
# driver.minimize_window()
driver.get(enlace)

In [7]:
resultados = driver.find_element(By.CLASS_NAME, "s-results__count")
print(resultados.text)

118 Resultados


extraccion de datos

In [78]:
cards = 'Hyundai Accent 2024\nAUTO NUEVO'
        #  'Hyundai Creta Grand 2025\nAUTO NUEVO',
        #  'Hyundai Tucson 2024\nAUTO NUEVO',
        #  'Hyundai Tucson 2024\nAUTO NUEVO',
        #  'Hyundai Creta Grand 2025\nAUTO NUEVO',
        #  'Ford F-150 2024\nAUTO NUEVO',
        #  'Hyundai Grand I10 2024\nAUTO NUEVO',
        #  'Hyundai Tucson 2025\nAUTO NUEVO'
         

tarejtas = cards.split("\n", 1)[0]
tarejtas

'Hyundai Accent 2024'

In [9]:

BRANDS = [
    "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
    "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
    "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
    "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
    "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen"
]

encabezado = driver.find_elements(By.CLASS_NAME, "c-results__header")

tiutlo_auto = []
marca_auto = []
modelo_auto = []
año_auto = []

for enc in encabezado:
    titulo = enc.text
    titulo_ = titulo.split("\n", 1)[0].strip()
    for brand in BRANDS:
        if brand in titulo_:
            
            año = titulo_.split()[-1]
            modelo = titulo_.replace(brand, "").replace(año, "").strip()

            tiutlo_auto.append(titulo_)
            marca_auto.append(brand)    
            modelo_auto.append(modelo)
            año_auto.append(año)
            
modelo_auto

['Cx-30',
 'Ranger',
 'Oroch',
 'Sportage',
 'Urvan',
 'Ranger Raptor',
 'Creta Grand',
 'Expedition',
 'Sportage',
 'Tucson',
 'Bronco Sport',
 'Grand I10',
 'Sportage']

In [36]:
import re

BRANDS = [
    "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
    "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
    "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
    "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
    "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen"
]
brand_re = re.compile(r'^(?P<brand>' + '|'.join(map(re.escape, sorted(BRANDS, key=len, reverse=True))) + r')\b', re.I)
year_re  = re.compile(r'((?:19|20)\d{2})\s*$')
KEEP_UPPER = {"BMW","BYD","GMC","MG","RAM","DFSK"}

def parse_card_title(raw: str):
    # 1) Quédate con la primera línea (títulos vienen como "… 2024\nAUTO NUEVO")
    title = raw.split("\n", 1)[0].strip()

    # 2) Año al final
    m_year = year_re.search(title)
    anio = int(m_year.group(1)) if m_year else None
    name = year_re.sub("", title).strip()

    # 3) Marca al inicio (incluye marcas de 2 palabras)
    m_brand = brand_re.match(name)
    if m_brand:
        brand_raw = m_brand.group("brand")
        marca = brand_raw if brand_raw.upper() in KEEP_UPPER else brand_raw.title()
        modelo = name[m_brand.end():].strip()
    else:
        parts = name.split(maxsplit=1)
        marca = (parts[0].upper() if parts and parts[0].upper() in KEEP_UPPER
                 else (parts[0].title() if parts else ""))
        modelo = parts[1] if len(parts) > 1 else ""

    return {"titulo": title, "marca": marca, "modelo": modelo, "anio": anio}

cards = ['Hyundai Accent 2024\nAUTO NUEVO',
         'Hyundai Creta Grand 2025\nAUTO NUEVO',
         'Hyundai Tucson 2024\nAUTO NUEVO',
         'Hyundai Tucson 2024\nAUTO NUEVO',
         'Hyundai Creta Grand 2025\nAUTO NUEVO',
         'Ford F-150 2024\nAUTO NUEVO',
         'Hyundai Grand I10 2024\nAUTO NUEVO',
         'Hyundai Tucson 2025\nAUTO NUEVO']

parsed = [parse_card_title(s) for s in cards]
for r in parsed:
    print(r)

{'titulo': 'Hyundai Accent 2024', 'marca': 'Hyundai', 'modelo': 'Accent', 'anio': 2024}
{'titulo': 'Hyundai Creta Grand 2025', 'marca': 'Hyundai', 'modelo': 'Creta Grand', 'anio': 2025}
{'titulo': 'Hyundai Tucson 2024', 'marca': 'Hyundai', 'modelo': 'Tucson', 'anio': 2024}
{'titulo': 'Hyundai Tucson 2024', 'marca': 'Hyundai', 'modelo': 'Tucson', 'anio': 2024}
{'titulo': 'Hyundai Creta Grand 2025', 'marca': 'Hyundai', 'modelo': 'Creta Grand', 'anio': 2025}
{'titulo': 'Ford F-150 2024', 'marca': 'Ford', 'modelo': 'F-150', 'anio': 2024}
{'titulo': 'Hyundai Grand I10 2024', 'marca': 'Hyundai', 'modelo': 'Grand I10', 'anio': 2024}
{'titulo': 'Hyundai Tucson 2025', 'marca': 'Hyundai', 'modelo': 'Tucson', 'anio': 2025}


In [37]:
precio = driver.find_elements(By.CLASS_NAME, "c-results-mount__price")
precio_auto = []
for prec in precio:
    costo = prec.text
    precio_auto.append(costo)
    
precio_auto[0]

'US$ 26,990'

In [38]:
tag =  driver.find_elements(By.CLASS_NAME, "c-results-tag__stick")
tag_ = []
for det in tag:
    descripcion = det.text
    tag_.append(descripcion)

tag_[0]

'PREMIUM'

In [39]:

detalles = driver.find_elements(By.CLASS_NAME, "c-results-details__description")

detalle_auto = []
combustible_auto = []
transmision_auto = []
caja_auto = []
kilometraje_auto = []
ubicacion_auto = []
descripcion_short_auto = []

detalles_autos = []

for detalle in detalles:
    
    detalle_text = detalle.text
    # print(info)
    
    # normaliza y separa líneas no vacías
    t = re.sub(r'[\r\xa0]', ' ', detalle_text).strip()
    lines = [l.strip() for l in t.splitlines() if l.strip()]

    # 1) combustible y transmisión (vienen en la 1ra línea separados por "|")
    combustible = transmision_raw = None
    if lines:
        p0 = re.split(r'\s*\|\s*', lines[0])  # acepta con o sin espacios: "Gasolina|Automática - Secuencial"
        combustible = p0[0].title() if p0 else None
        transmision_raw = p0[1] if len(p0) > 1 else None

    # 1.1) separa transmisión en tipo y caja si viene "Automática - Secuencial"
    tipo_transmision = caja = None
    if transmision_raw:
        parts = [p.strip() for p in transmision_raw.split("-")]
        tipo_transmision = parts[0]
        caja = parts[1] if len(parts) > 1 else None

    # 2) kilometraje (busca en todo el texto)
    km_m = re.search(r'(\d[\d.,]*)\s*kms?', t, flags=re.I)
    kilometraje_km = int(re.sub(r'[^\d]', '', km_m.group(1))) if km_m else None

    # 3) ubicación (primera línea con coma que no sea "Kms")
    ubicacion = next((l for l in lines if ',' in l and 'km' not in l.lower()), None)

    # 4) “modelo/etiqueta” (última línea)
    modelo_line = lines[-1] if lines else None
    
    # salidas
    detalle_auto.append(detalle_text)
    combustible_auto.append(combustible)
    transmision_auto.append(tipo_transmision)
    caja_auto.append(caja)
    kilometraje_auto.append(kilometraje_km)
    ubicacion_auto.append(ubicacion)
    
    detalles_autos.append({
        "detalle": detalle_text,
        "combustible": combustible,
        "transmision": tipo_transmision,
        "caja": caja,
        "kilometraje_km": kilometraje_km,
        "ubicacion": ubicacion,
        "descripcion_short": modelo_line
    })
#     descripcion_short_auto.append(modelo_line)

detalles_autos

[{'detalle': 'Gasolina|Mecánica\nLima, Lima\nCompra Tu Próximo 0KM En Maquinarias Mazda',
  'combustible': 'Gasolina',
  'transmision': 'Mecánica',
  'caja': None,
  'kilometraje_km': 0,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Compra Tu Próximo 0KM En Maquinarias Mazda'},
 {'detalle': 'Gasolina|Mecánica\nLima, Lima\nCompra Tu Próximo 0KM En Maquinarias DFSK',
  'combustible': 'Gasolina',
  'transmision': 'Mecánica',
  'caja': None,
  'kilometraje_km': 0,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Compra Tu Próximo 0KM En Maquinarias DFSK'},
 {'detalle': 'Gasolina|Automática\nLima, Lima\nCompra Tu Próximo 0KM En Maquinarias Hyundai',
  'combustible': 'Gasolina',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje_km': 0,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Compra Tu Próximo 0KM En Maquinarias Hyundai'},
 {'detalle': 'Gasolina|Automática\nLima, Lima\nENTREGA INMEDIATA VERSIÓN SPORT',
  'combustible': 'Gasolina',
  'transmision': 'Automáti

In [42]:
enlaces = []
enlace = driver.find_elements(By.CLASS_NAME, "c-results__link")   
for link in enlace:
    url = link.get_attribute("href")    
    enlaces.append(url)

enlaces

['https://neoauto.com/auto/nuevo/mazda-3-2025-1705780',
 'https://neoauto.com/auto/nuevo/dfsk-c35-2025-1827085',
 'https://neoauto.com/auto/nuevo/hyundai-stargazer-2025-1827060',
 'https://neoauto.com/auto/nuevo/hyundai-accent-2025-1846395',
 'https://neoauto.com/auto/nuevo/dfsk-e5-2025-1827084',
 'https://neoauto.com/auto/nuevo/honda-cr-v-2025-1709635',
 'https://neoauto.com/auto/nuevo/jmc-jim-remax-4x4-2026-1842713',
 'https://neoauto.com/auto/nuevo/ford-explorer-2025-1706299',
 'https://neoauto.com/auto/nuevo/kia-sportage-2025-1846416',
 'https://neoauto.com/auto/nuevo/jmc-grand-avenue-luxury-2026-1842722',
 'https://neoauto.com/auto/nuevo/honda-city-2024-1843626',
 'https://neoauto.com/auto/nuevo/jmc-grand-avenue-luxury-2026-1842721',
 'https://neoauto.com/auto/nuevo/honda-civic-2024-1709637',
 'https://neoauto.com/auto/nuevo/nissan-patrol-2024-1709644',
 'https://neoauto.com/auto/nuevo/renault-master-furgon-2026-1827080',
 'https://neoauto.com/auto/nuevo/ford-maverick-2024-1726273

## Extraccion de Informacion de segun el tipo de Vehicuklo y Condicion

Es necesario preguntar si quiere un auto, nuevo, usado o semenuevo.

In [10]:
while True:
    tipo = input("Es necesario preguntar si quiere un auto, autos (1), motos (2) o camiones (3)")

    if tipo == "1":
        tipo_vehiculo = "autos"
        break
    elif tipo == "2":
        tipo_vehiculo = "motos"
        break
    elif tipo == "3":
        tipo_vehiculo = "camiones"  
        break  
    
# while True:
#     tipo = input("Es necesario preguntar si quiere un auto que tipo de auto: camioneta (1), sedan (2), hatchback (3), pick-up (4), vans (5)")

#     if tipo == "1":
#         categoria = "camionetas-suv"
#         break
#     elif tipo == "2":
#         categoria = "sedan"
#         break
#     elif tipo == "3":
#         categoria = "hatchback" 
#         break  
#     elif tipo == "4":
#         categoria = "pick-up" 
#         break  
#     elif tipo == "5":
#         categoria = "vans" 
#         break  
#     elif tipo == "6":
#         categoria = "deportivo" 
#         break  


while True:
    tipo = input("Es necesario preguntar si quiere un auto, nuevo (1), usados (2) o seminuevos (3)")

    unidad = ""
    if tipo == "1":
        unidad = "nuevos"
        break
    elif tipo == "2":
        unidad = "usados"
        break
    elif tipo == "3":
        unidad = "seminuevos"
        break  

print(f"Se extraerán datos de: {tipo_vehiculo} - {unidad}")

Se extraerán datos de: autos - nuevos


Construccion de enlace

## Extraccion de Datos

In [11]:
# usados = range(1, 182)
# nuevo = range(1, 6)
# seminuevos = range(1, 44)

paginas = []

if unidad == "usados":
    for i in  range(1, 182):
        url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}?page={i}"
        paginas.append({
            "url": url,
            "tipo": unidad,
            "tipo_vehiculo": tipo_vehiculo
        })

if unidad == "nuevos":
    for i in  range(1, 7):
        url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}?page={i}"
        paginas.append({
            "url": url,
            "tipo": unidad,
            "tipo_vehiculo": tipo_vehiculo
        })

if unidad == "seminuevos":
    for i in  range(1, 44):
        url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}?page={i}"
        paginas.append({
            "url": url,
            "tipo": unidad,
            "tipo_vehiculo": tipo_vehiculo
        })


# paginas[:10]
print(f"Se extraerán datos de {len(paginas)} páginas")

Se extraerán datos de 6 páginas


In [12]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")   # modo oculto
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
# driver.minimize_window()


neo_autos = []

for enlace in tqdm(paginas, desc="Páginas procesadas", unit="vehiculo"):
    
    tipo = enlace["tipo"]
    link_pagina = enlace["url"]
    tipo_vehiculo = enlace["tipo_vehiculo"]
    
    driver.get(link_pagina)
    
    BRANDS = sorted([
        "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
        "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
        "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
        "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
        "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen"
    ], key=len, reverse=True)


    ## NOMBRE, MARCA, MODELO, AÑO
    
    encabezado = driver.find_elements(By.CLASS_NAME, "c-results__header")

    titulo_auto = []
    marca_auto = []
    modelo_auto = []
    año_auto = []

    for enc in encabezado:
        titulo = enc.text
        titulo_ = titulo.split("\n", 1)[0].strip()
        for brand in BRANDS:
            if brand in titulo_:
                año = titulo_.split()[-1]
                modelo = titulo_.replace(brand, "").replace(año, "").strip()

                titulo_auto.append(titulo_)
                marca_auto.append(brand)    
                modelo_auto.append(modelo)
                año_auto.append(año)
                
    
    ## PRECIO
    
    precio = driver.find_elements(By.CLASS_NAME, "c-results-mount__price")
    precio_auto = []
    for prec in precio:
        costo = prec.text
        precio_auto.append(costo)

    precio_auto[0]
    
    ## TAGS
    
    tag =  driver.find_elements(By.CLASS_NAME, "c-results-tag__stick")
    tags = []
    for det in tag:
        descripcion = det.text
        tags.append(descripcion)
    
    ## DETALLE DE AUTO
        
    detalles = driver.find_elements(By.CLASS_NAME, "c-results-details__description")

    detalle_auto = []
    combustible_auto = []
    transmision_auto = []
    caja_auto = []
    kilometraje_auto = []
    ubicacion_auto = []

    for detalle in detalles:

        detalle_text = detalle.text

        # normaliza y separa líneas no vacías
        t = re.sub(r'[\r\xa0]', ' ', detalle_text).strip()
        lines = [l.strip() for l in t.splitlines() if l.strip()]

        # 1) combustible y transmisión (vienen en la 1ra línea separados por "|")
        combustible = transmision_raw = None
        if lines:
            p0 = re.split(r'\s*\|\s*', lines[0])  # acepta con o sin espacios: "Gasolina|Automática - Secuencial"
            combustible = p0[0].title() if p0 else None
            transmision_raw = p0[1] if len(p0) > 1 else None

        # 1.1) separa transmisión en tipo y caja si viene "Automática - Secuencial"
        tipo_transmision = caja = None
        if transmision_raw:
            parts = [p.strip() for p in transmision_raw.split("-")]
            tipo_transmision = parts[0]
            caja = parts[1] if len(parts) > 1 else None

        # 2) kilometraje (busca en todo el texto)
        km_m = re.search(r'(\d[\d.,]*)\s*kms?', t, flags=re.I)
        kilometraje_km = int(re.sub(r'[^\d]', '', km_m.group(1))) if km_m else None

        # 3) ubicación (primera línea con coma que no sea "Kms")
        ubicacion = next((l for l in lines if ',' in l and 'km' not in l.lower()), None)

        # 4) “modelo/etiqueta” (última línea)
        modelo_line = lines[-1] if lines else None

        detalle_auto.append(detalle_text)
        combustible_auto.append(combustible)
        transmision_auto.append(tipo_transmision)
        caja_auto.append(caja)
        kilometraje_auto.append(kilometraje_km)
        ubicacion_auto.append(ubicacion)
    
    ## URL DE AUTO
    
    url_auto = []
    enlace = driver.find_elements(By.CLASS_NAME, "c-results__link")   
    for link in enlace:
        url = link.get_attribute("href")    
        url_auto.append(url)
    
    ## ALMACENAMIENTO DE DATOS
    
    zip_list = zip(titulo_auto, marca_auto, modelo_auto, año_auto, precio_auto, tags, detalle_auto, combustible_auto, transmision_auto, caja_auto, kilometraje_auto, ubicacion_auto, url_auto)
    
    data_auto_page = []
    for titulo_auto_, marca_auto_, modelo_auto_, año_auto_, precio_auto_, tags_, detalle_auto_, combustible_auto_, transmision_auto_, caja_auto_, kilometraje_auto_, ubicacion_auto_, url_auto_ in zip_list:
        data_auto_page.append({
            "titulo": titulo_auto_,
            "tipo_vehiculo": tipo_vehiculo,
            "marca": marca_auto_,
            "modelo": modelo_auto_,
            "año": año_auto_,
            "precio": precio_auto_,
            "detalle": detalle_auto_,
            "combustible": combustible_auto_,
            "transmision": transmision_auto_,
            "caja": caja_auto_,
            "kilometraje": kilometraje_auto_,
            "ubicacion": ubicacion_auto_,
            "url_auto":url_auto_,
            "tags": tags_,
            "tipo": tipo,
            "pagina": link_pagina
        })
        
    neo_autos.extend(data_auto_page)
    

# driver.quit()
    
# print(f"Se extrajeorn informacion de {len(neo_autos)} unidades")
    

Páginas procesadas: 100%|██████████| 6/6 [00:15<00:00,  2.61s/vehiculo]


In [14]:
print(f"Se extrajeorn informacion de {len(neo_autos)} unidades")

Se extrajeorn informacion de 70 unidades


In [35]:
neo_autos

[{'titulo': 'Hyundai Accent 2024',
  'tipo_vehiculo': 'autos',
  'marca': 'Hyundai',
  'modelo': 'Accent',
  'año': '2024',
  'precio': 'US$ 17,990',
  'detalle': 'Gas GLP|Automática\nLima, Lima\nGARANTIA VIGENTE NEW ACCENT FULL GLP',
  'combustible': 'Gas Glp',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje': None,
  'ubicacion': 'Lima, Lima',
  'url_auto': 'https://neoauto.com/auto/nuevo/hyundai-accent-2024-1846381',
  'tags': 'PREMIUM',
  'tipo': 'nuevos',
  'pagina': 'https://neoauto.com/venta-de-autos-nuevos?page=1'},
 {'titulo': 'Hyundai Creta Grand 2025',
  'tipo_vehiculo': 'autos',
  'marca': 'Hyundai',
  'modelo': 'Creta Grand',
  'año': '2025',
  'precio': 'US$ 25,990',
  'detalle': 'Gas GLP|Automática\nLima, Lima\nGARANTÍA VIGENTE CRETA GRAND FULL GLP 2025',
  'combustible': 'Gas Glp',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje': None,
  'ubicacion': 'Lima, Lima',
  'url_auto': 'https://neoauto.com/auto/nuevo/hyundai-creta-grand-2025-1846397

In [33]:
neo_autos[10]

{'titulo': 'Hyundai Santa Fe 2025',
 'tipo_vehiculo': 'autos',
 'marca': 'Hyundai',
 'modelo': 'Santa Fe',
 'año': '2025',
 'precio': 'US$ 43,990',
 'detalle': 'Gasolina|Automática\nLima, Lima\nCompra Tu Próximo 0KM En Maquinarias Honda',
 'combustible': 'Gasolina',
 'transmision': 'Automática',
 'caja': None,
 'kilometraje': 0,
 'ubicacion': 'Lima, Lima',
 'url_auto': 'https://neoauto.com/auto/nuevo/honda-accord-2025-1827043',
 'tags': 'PREMIUM',
 'tipo': 'nuevos',
 'pagina': 'https://neoauto.com/venta-de-autos-nuevos?page=2'}

In [15]:
neo_autos_df = pd.DataFrame(neo_autos)
neo_autos_df

,titulo,tipo_vehiculo,marca,modelo,año,precio,detalle,combustible,transmision,caja,kilometraje,ubicacion,url_auto,tags,tipo,pagina
0,Hyundai Accent 2024,autos,Hyundai,Accent,2024,"US$ 17,990","Gas GLP|Automática\nLima, Lima\nGARANTIA VIGEN...",Gas Glp,Automática,None,NaN,"Lima, Lima",https://neoauto.com/auto/nuevo/hyundai-accent-...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
1,Hyundai Creta Grand 2025,autos,Hyundai,Creta Grand,2025,"US$ 25,990","Gas GLP|Automática\nLima, Lima\nGARANTÍA VIGEN...",Gas Glp,Automática,None,NaN,"Lima, Lima",https://neoauto.com/auto/nuevo/hyundai-creta-g...,MOTOR POTENTE,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
2,Hyundai Tucson 2024,autos,Hyundai,Tucson,2024,"US$ 29,990","Gasolina|Automática - Secuencial\nLima, Lima\n...",Gasolina,Automática,Secuencial,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/hyundai-tucson-...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
3,Hyundai I20 2024,autos,Hyundai,I20,2024,"US$ 16,490","Gas GLP|Mecánica\nLima, Lima\nENTREGA INMEDIAT...",Gas Glp,Mecánica,None,NaN,"Lima, Lima",https://neoauto.com/auto/nuevo/hyundai-i20-202...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2
4,Hyundai Tucson 2024,autos,Hyundai,Tucson,2024,"US$ 29,990","Gasolina|Automática - Secuencial\nLima, Lima\n...",Gasolina,Automática,Secuencial,NaN,"Lima, Lima",https://neoauto.com/auto/nuevo/hyundai-tucson-...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Kia Sportage 2025,autos,Kia,Sportage,2025,"US$ 29,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/dfsk-e5-2025-18...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6
66,Nissan Patrol 2024,autos,Nissan,Patrol,2024,"US$ 41,490","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/honda-cr-v-2025...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6
67,Renault Master Furgon 2026,autos,Renault,Master Furgon,2026,"US$ 17,990","Diesel|Mecánica\nLima, Lima\nPotencia Y Respal...",Diesel,Mecánica,None,NaN,"Lima, Lima",https://neoauto.com/auto/nuevo/jmc-jim-remax-4...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6
68,Ford Maverick 2024,autos,Ford,Maverick,2024,"US$ 51,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/ford-explorer-2...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6


In [17]:
neo_autos_df[["titulo", "marca", "modelo", "año", "precio", "url_auto"]]

,titulo,marca,modelo,año,precio,url_auto
0,Hyundai Accent 2024,Hyundai,Accent,2024,"US$ 17,990",https://neoauto.com/auto/nuevo/hyundai-accent-...
1,Hyundai Creta Grand 2025,Hyundai,Creta Grand,2025,"US$ 25,990",https://neoauto.com/auto/nuevo/hyundai-creta-g...
2,Hyundai Tucson 2024,Hyundai,Tucson,2024,"US$ 29,990",https://neoauto.com/auto/nuevo/hyundai-tucson-...
3,Hyundai I20 2024,Hyundai,I20,2024,"US$ 16,490",https://neoauto.com/auto/nuevo/hyundai-i20-202...
4,Hyundai Tucson 2024,Hyundai,Tucson,2024,"US$ 29,990",https://neoauto.com/auto/nuevo/hyundai-tucson-...
...,...,...,...,...,...,...
65,Kia Sportage 2025,Kia,Sportage,2025,"US$ 29,990",https://neoauto.com/auto/nuevo/dfsk-e5-2025-18...
66,Nissan Patrol 2024,Nissan,Patrol,2024,"US$ 41,490",https://neoauto.com/auto/nuevo/honda-cr-v-2025...
67,Renault Master Furgon 2026,Renault,Master Furgon,2026,"US$ 17,990",https://neoauto.com/auto/nuevo/jmc-jim-remax-4...
68,Ford Maverick 2024,Ford,Maverick,2024,"US$ 51,990",https://neoauto.com/auto/nuevo/ford-explorer-2...


In [18]:
# normalizaciones básicas
neo_autos_df["url_low"]     = neo_autos_df["url_auto"].str.lower()
neo_autos_df["marca_slug"]  = neo_autos_df["marca"].str.lower().str.replace(r"[^a-z0-9]+","-", regex=True)
neo_autos_df["modelo_slug"] = neo_autos_df["modelo"].str.lower().str.replace(r"[^a-z0-9]+","-", regex=True)

# checks fila a fila (sin regex fancy)
neo_autos_df["brand_in_url"] = neo_autos_df.apply(lambda r: r["marca_slug"]  in r["url_low"], axis=1)
neo_autos_df["model_in_url"] = neo_autos_df.apply(lambda r: r["modelo_slug"] in r["url_low"], axis=1)
neo_autos_df["year_in_url"]  = neo_autos_df.apply(lambda r: str(r["año"]) in r["url_low"], axis=1)  # usa "anio" si tu col se llama así

neo_autos_df["url_ok"] = neo_autos_df[["brand_in_url","model_in_url","year_in_url"]].all(axis=1)

In [25]:
neo_autos_df

,titulo,tipo_vehiculo,marca,modelo,año,precio,detalle,combustible,transmision,caja,...,tags,tipo,pagina,url_low,marca_slug,modelo_slug,brand_in_url,model_in_url,year_in_url,url_ok
0,Hyundai Accent 2024,autos,Hyundai,Accent,2024,"US$ 17,990","Gas GLP|Automática\nLima, Lima\nGARANTIA VIGEN...",Gas Glp,Automática,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1,https://neoauto.com/auto/nuevo/hyundai-accent-...,hyundai,accent,True,True,True,True
1,Hyundai Creta Grand 2025,autos,Hyundai,Creta Grand,2025,"US$ 25,990","Gas GLP|Automática\nLima, Lima\nGARANTÍA VIGEN...",Gas Glp,Automática,None,...,MOTOR POTENTE,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1,https://neoauto.com/auto/nuevo/hyundai-creta-g...,hyundai,creta-grand,True,True,True,True
2,Hyundai Tucson 2024,autos,Hyundai,Tucson,2024,"US$ 29,990","Gasolina|Automática - Secuencial\nLima, Lima\n...",Gasolina,Automática,Secuencial,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1,https://neoauto.com/auto/nuevo/hyundai-tucson-...,hyundai,tucson,True,True,True,True
3,Hyundai I20 2024,autos,Hyundai,I20,2024,"US$ 16,490","Gas GLP|Mecánica\nLima, Lima\nENTREGA INMEDIAT...",Gas Glp,Mecánica,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2,https://neoauto.com/auto/nuevo/hyundai-i20-202...,hyundai,i20,True,True,True,True
4,Hyundai Tucson 2024,autos,Hyundai,Tucson,2024,"US$ 29,990","Gasolina|Automática - Secuencial\nLima, Lima\n...",Gasolina,Automática,Secuencial,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2,https://neoauto.com/auto/nuevo/hyundai-tucson-...,hyundai,tucson,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Kia Sportage 2025,autos,Kia,Sportage,2025,"US$ 29,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6,https://neoauto.com/auto/nuevo/dfsk-e5-2025-18...,kia,sportage,False,False,True,False
66,Nissan Patrol 2024,autos,Nissan,Patrol,2024,"US$ 41,490","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6,https://neoauto.com/auto/nuevo/honda-cr-v-2025...,nissan,patrol,False,False,False,False
67,Renault Master Furgon 2026,autos,Renault,Master Furgon,2026,"US$ 17,990","Diesel|Mecánica\nLima, Lima\nPotencia Y Respal...",Diesel,Mecánica,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6,https://neoauto.com/auto/nuevo/jmc-jim-remax-4...,renault,master-furgon,False,False,True,False
68,Ford Maverick 2024,autos,Ford,Maverick,2024,"US$ 51,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6,https://neoauto.com/auto/nuevo/ford-explorer-2...,ford,maverick,True,False,False,False


In [ ]:
neo_autos_df[neo_autos_df["titulo"]=="Hyundai Creta 2026"][["titulo", "precio", "url_auto", "pagina"]]

,titulo,precio,url_auto,pagina
14,Hyundai Creta 2026,"US$ 31,660",https://neoauto.com/auto/nuevo/nissan-x-trail-...,https://neoauto.com/venta-de-autos-nuevos?page=2


In [ ]:
# ver sospechosos
sospechosos = neo_autos_df.loc[~neo_autos_df["url_ok"], ["titulo","marca","modelo","año","url_auto","brand_in_url","model_in_url","year_in_url"]]
sospechosos

,titulo,marca,modelo,año,url_auto,brand_in_url,model_in_url,year_in_url
10,Hyundai Santa Fe 2025,Hyundai,Santa Fe,2025,https://neoauto.com/auto/nuevo/honda-accord-20...,False,False,True
11,Mazda Cx-5 2026,Mazda,Cx-5,2026,https://neoauto.com/auto/nuevo/hyundai-santa-f...,False,False,False
12,Renault Stepway 2025,Renault,Stepway,2025,https://neoauto.com/auto/nuevo/mazda-cx-5-2026...,False,False,False
13,Nissan X-Trail 2025,Nissan,X-Trail,2025,https://neoauto.com/auto/nuevo/renault-stepway...,False,False,True
14,Hyundai Creta 2026,Hyundai,Creta,2026,https://neoauto.com/auto/nuevo/nissan-x-trail-...,False,False,False
15,Mazda Cx-9 2025,Mazda,Cx-9,2025,https://neoauto.com/auto/nuevo/hyundai-creta-2...,False,False,False
16,Mazda 2 Sport 2025,Mazda,2 Sport,2025,https://neoauto.com/auto/nuevo/mazda-cx-9-2025...,True,False,True
17,Nissan Versa 2025,Nissan,Versa,2025,https://neoauto.com/auto/nuevo/mazda-2-sport-2...,False,False,True
18,Mazda 2 2024,Mazda,2,2024,https://neoauto.com/auto/nuevo/dfsk-glory-560-...,False,True,False
20,Ford Ranger 2025,Ford,Ranger,2025,https://neoauto.com/auto/nuevo/dfsk-glory-330s...,False,False,True
